In [1]:
from IPython.display import  HTML
from dataset import TUBerlinDataset
from prepare_data import convert_and_quantize_svg

labels = [ "backpack" ]
dataset = TUBerlinDataset(labels)

for svg_content in dataset[0:1]:
    display(HTML(f'<div style="width: 200px; background-color: white;"> {svg_content} </div>'))
    print(len(svg_content))

    output = convert_and_quantize_svg(svg_content, bins=256)
    display(HTML(f'<div style="width: 200px; background-color: white;"> {output} </div>'))
    print(len(output))

Loading TUBerlin files: 100%|██████████| 1/1 [00:00<00:00, 131.42it/s]


4341


2461


In [2]:
from IPython.display import  HTML
from dataset import QuickDrawDataset
from prepare_data import convert_and_quantize_svg

labels = [ "bulldozer" ]
dataset = QuickDrawDataset(labels)

for svg_content in dataset[0:1]:
    display(HTML(f'<div style="width: 256px; background-color: white;"> {svg_content} </div>'))
    print(len(svg_content))

    output = convert_and_quantize_svg(svg_content, bins=256)
    display(HTML(f'<div style="width: 200px; background-color: white;"> {output} </div>'))
    print(len(output))

Loading QuickDraw files: 100%|██████████| 1/1 [00:05<00:00,  5.30s/it]


1609


1036


In [3]:
from IPython.display import  HTML
from utils import svg_to_tensor, tensor_to_svg

original_svg = """<svg xmlns="http://www.w3.org/2000/svg" width="256" height="256" viewBox="0 0 256 256"><g stroke-width="0.6">
<path d="M 26 22 L 32 50 L 33 84" stroke="black" fill="none" stroke-linecap="round" stroke-linejoin="round"/>
<path d="M 24 24 L 56 16 L 114 7 L 113 75 L 111 82 L 105 86" stroke="black" fill="none" stroke-linecap="round" stroke-linejoin="round"/>
<path d="M 8 101 L 26 96 L 130 84" stroke="black" fill="none" stroke-linecap="round" stroke-linejoin="round"/>
<path d="M 7 103 L 6 153 L 11 156 L 24 157 L 59 150 L 127 147" stroke="black" fill="none" stroke-linecap="round" stroke-linejoin="round"/>
<path d="M 138 86 L 132 142 L 134 148" stroke="black" fill="none" stroke-linecap="round" stroke-linejoin="round"/>
<path d="M 133 85 L 133 78 L 157 18 L 173 0 L 189 24 L 203 57 L 207 91 L 196 106 L 195 126 L 203 134 L 214 138 L 229 139 L 237 136 L 247 128 L 240 105 L 226 89 L 209 87 L 204 92" stroke="black" fill="none" stroke-linecap="round" stroke-linejoin="round"/>
<path d="M 202 140 L 209 147 L 211 146 L 215 127 L 221 141 L 225 140 L 232 130 L 238 137 L 247 124 L 255 121" stroke="black" fill="none" stroke-linecap="round" stroke-linejoin="round"/>
<path d="M 201 141 L 201 141" stroke="black" fill="none" stroke-linecap="round" stroke-linejoin="round"/>
<path d="M 200 128 L 204 137" stroke="black" fill="none" stroke-linecap="round" stroke-linejoin="round"/>
<path d="M 2 163 L 38 150 L 155 137 L 163 139 L 166 149 L 166 156 L 158 166 L 148 173 L 121 180 L 41 191 L 23 191 L 7 185 L 1 176 L 1 160 L 14 152" stroke="black" fill="none" stroke-linecap="round" stroke-linejoin="round"/>
</g></svg>
        """
tensor = svg_to_tensor(original_svg)
reconstructed_svg = tensor_to_svg(tensor)

print("original svg")
display(HTML(f'<div style="width: 256px; background-color: white;"> {original_svg} </div>'))

print("svg -> tensor -> svg")
display(HTML(f'<div style="width: 256px; background-color: white;"> {reconstructed_svg} </div>'))

original svg


svg -> tensor -> svg
